In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON, XML
from pprint import pprint
import pandas as pd

In [2]:
def querySparql(query):
    # The ld.stadt-zuerich.ch endpoints doesn't seem to work, use LINDA instead
    #sparql = SPARQLWrapper("https://ld.stadt-zuerich.ch/query")
    sparql = SPARQLWrapper("https://lindas-data.ch:8443/lindas/query")
    sparql.setHTTPAuth("BASIC")
    sparql.setCredentials("public", "public")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.queryAndConvert()
    return results['results']['bindings']

In [3]:
allDatasets = """
PREFIX qb: <http://purl.org/linked-data/cube#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?dataset ?label WHERE { GRAPH <https://linked.opendata.swiss/graph/zh/statistics> {

   ?dataset a qb:DataSet ; 
      rdfs:label ?label .     
    
   # ?obs a qb:Observation ;
   ?obs <http://purl.org/linked-data/cube#dataSet> ?dataset .
   
   # FILTER (?dataset = <https://ld.stadt-zuerich.ch/statistics/dataset/AVA-RAUM-ZEIT-GGH-HEL-SEX>)  
}}
GROUP BY ?dataset ?label
LIMIT 1000
"""
print(allDatasets)


PREFIX qb: <http://purl.org/linked-data/cube#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?dataset ?label WHERE { GRAPH <https://linked.opendata.swiss/graph/zh/statistics> {

   ?dataset a qb:DataSet ; 
      rdfs:label ?label .     
    
   # ?obs a qb:Observation ;
   ?obs <http://purl.org/linked-data/cube#dataSet> ?dataset .
   
   # FILTER (?dataset = <https://ld.stadt-zuerich.ch/statistics/dataset/AVA-RAUM-ZEIT-GGH-HEL-SEX>)  
}}
GROUP BY ?dataset ?label
LIMIT 1000



In [4]:
datasetResult = querySparql(allDatasets)

In [5]:
datasets = [{'uri': d['dataset']['value'], 'label': d['label']['value']} for d in datasetResult]
datasets

[{'label': u'Arbeitsst\xe4tten nach Betriebsart, Raum, Zeit',
  'uri': u'https://ld.stadt-zuerich.ch/statistics/dataset/AST-RAUM-ZEIT-BTA'},
 {'label': u'Besch\xe4ftigte nach Betriebsart, Raum, Geschlecht, Zeit',
  'uri': u'https://ld.stadt-zuerich.ch/statistics/dataset/BES-RAUM-ZEIT-BTA-SEX'},
 {'label': u'Besch\xe4ftigte nach Betriebsart, Raum, Zeit',
  'uri': u'https://ld.stadt-zuerich.ch/statistics/dataset/BES-RAUM-ZEIT-BTA'},
 {'label': u'Zuschauer/innen, Besucher/innen nach Betriebsart, Heimatland, Raum, Zeit',
  'uri': u'https://ld.stadt-zuerich.ch/statistics/dataset/ZUS-RAUM-ZEIT-BTA-HEL'},
 {'label': u'Zuschauer/innen, Besucher/innen nach Betriebsart, Raum, Geschlecht, Zeit',
  'uri': u'https://ld.stadt-zuerich.ch/statistics/dataset/ZUS-RAUM-ZEIT-BTA-SEX'},
 {'label': u'Zuschauer/innen, Besucher/innen nach Betriebsart, Raum, Zeit',
  'uri': u'https://ld.stadt-zuerich.ch/statistics/dataset/ZUS-RAUM-ZEIT-BTA'},
 {'label': u'Sch\xfcler/innen und Student/innen nach Betriebsart, Ra

In [6]:
def generateMetaQuery(dataset):
    metaQuery = u'''
    PREFIX qb: <http://purl.org/linked-data/cube#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    SELECT ?dataset ?title ?categoryLabel ?quelleLabel ?zeit ?updateDate ?glossarLabel ?btaLabel ?raumLabel
    WHERE { GRAPH <https://linked.opendata.swiss/graph/zh/statistics> {
       ?dataset a qb:DataSet ; 
            rdfs:label ?title .     

        # group
       OPTIONAL {
           ?category a <https://ld.stadt-zuerich.ch/schema/Category> ;
           rdfs:label ?categoryLabel ;
           skos:narrower* ?dataset .
        }

         # quelle, zeit, aktualisierungsdatum
        ?obs <http://purl.org/linked-data/cube#dataSet> ?dataset .
        OPTIONAL {
            ?obs <https://ld.stadt-zuerich.ch/statistics/attribute/QUELLE> ?quelle .
            ?quelle rdfs:label ?quelleLabel .
        }
        OPTIONAL {
            ?obs <https://ld.stadt-zuerich.ch/statistics/property/RAUM> ?raum .
            ?raum rdfs:label ?raumLabel .
        }
        OPTIONAL { ?obs <https://ld.stadt-zuerich.ch/statistics/property/ZEIT> ?zeit } .
        OPTIONAL { ?obs <https://ld.stadt-zuerich.ch/statistics/attribute/DATENSTAND> ?updateDate } .
        
        # evtl. GLOSSAR und BTA (und weitere) für Tags verwenden
        OPTIONAL {
            ?obs <https://ld.stadt-zuerich.ch/statistics/attribute/GLOSSAR> ?glossar .
            ?glossar rdfs:label ?glossarLabel .
        }
         OPTIONAL {
             ?obs <https://ld.stadt-zuerich.ch/statistics/property/BTA> ?bta .        
             ?bta rdfs:label ?btaLabel .
        }

       FILTER (?dataset = <DATASET_URI>)
    }}
    GROUP BY ?dataset ?title ?categoryLabel ?quelleLabel ?zeit ?updateDate ?glossarLabel ?btaLabel ?raumLabel
    LIMIT 1000'''
    metaQuery = metaQuery.replace('DATASET_URI', dataset['uri'])
    return metaQuery

def mapMetadata(dataset, metaResult):
    meta = {
        'uri': dataset['uri'],
        'title': dataset['label'],
        'description': '',
        'category': list(set([t.get('categoryLabel', {}).get('value', '') for t in metaResult])),
        'legal_basis': '',
        'spatial': list(set([t.get('raumLabel', {}).get('value', '') for t in metaResult])),
        'publisher': 'Open Data Zurich',
        'source': list(set([t.get('quelleLabel', {}).get('value', '') for t in metaResult])),
        'temporal': list(set([t.get('zeit', {}).get('value', '') for t in metaResult])),
        'quality': '',
        'published_date': list(set([t.get('zeit', {}).get('value', '') for t in metaResult])),
        'update_date': list(set([t.get('updateDate', {}).get('value', '') for t in metaResult])),
        'data_type': 'Datenaggregat',
        'periodicity': '',
        'tags': list(set([t.get('glossarLabel', {}).get('value', '') for t in metaResult])),
        'version': '',
        'remarks': '',
    }
    return meta

In [7]:
metadata = []
# only loop over first 5 datasets
for dataset in datasets[:5]:
    metaQuery = generateMetaQuery(dataset)
    # print(metaQuery)
    
    metaResult = querySparql(metaQuery)
    #pprint(metaResult[0])
    
    meta = mapMetadata(dataset, metaResult)
    pprint(meta)
    metadata.append(meta)

{'category': [''],
 'data_type': 'Datenaggregat',
 'description': '',
 'legal_basis': '',
 'periodicity': '',
 'published_date': [u'1944-12-31',
                    u'1981-12-31',
                    u'1994-12-31',
                    u'2006-12-31',
                    u'2001-12-31',
                    u'1978-06-30',
                    u'1949-12-31',
                    u'2014-06-30',
                    u'1997-12-31',
                    u'1974-12-31',
                    u'1957-12-31',
                    u'1976-12-31',
                    u'2005-12-31',
                    u'1966-12-31',
                    u'1940-12-31',
                    u'1990-12-31',
                    u'1946-12-31',
                    u'1970-12-31',
                    u'1961-12-31',
                    u'2016-12-31',
                    u'1993-12-31',
                    u'1964-12-31',
                    u'1974-06-30',
                    u'1988-12-31',
                    u'2017-12-31',
               

{'category': [u'Bildung, Kultur & Sport'],
 'data_type': 'Datenaggregat',
 'description': '',
 'legal_basis': '',
 'periodicity': '',
 'published_date': [u'2014-06-30',
                    u'1986-06-30',
                    u'1980-06-30',
                    u'1972-06-30',
                    u'2006-06-30',
                    u'1978-06-30',
                    u'1969-06-30',
                    u'2013-06-30',
                    u'1974-06-30',
                    u'2002-06-30',
                    u'2004-06-30',
                    u'1966-06-30',
                    u'1977-06-30',
                    u'1988-06-30',
                    u'2007-06-30',
                    u'1997-06-30',
                    u'2009-06-30',
                    u'1984-06-30',
                    u'1999-06-30',
                    u'1970-06-30',
                    u'2008-06-30',
                    u'2015-06-30',
                    u'1996-06-30',
                    u'1998-06-30',
                    u'2000

In [8]:
pprint(metadata)

[{'category': [''],
  'data_type': 'Datenaggregat',
  'description': '',
  'legal_basis': '',
  'periodicity': '',
  'published_date': [u'1944-12-31',
                     u'1981-12-31',
                     u'1994-12-31',
                     u'2006-12-31',
                     u'2001-12-31',
                     u'1978-06-30',
                     u'1949-12-31',
                     u'2014-06-30',
                     u'1997-12-31',
                     u'1974-12-31',
                     u'1957-12-31',
                     u'1976-12-31',
                     u'2005-12-31',
                     u'1966-12-31',
                     u'1940-12-31',
                     u'1990-12-31',
                     u'1946-12-31',
                     u'1970-12-31',
                     u'1961-12-31',
                     u'2016-12-31',
                     u'1993-12-31',
                     u'1964-12-31',
                     u'1974-06-30',
                     u'1988-12-31',
                     

In [9]:
pd.DataFrame(data=metadata)

,category,data_type,description,legal_basis,periodicity,published_date,publisher,quality,remarks,source,spatial,tags,temporal,title,update_date,uri,version
0,[],Datenaggregat,,,,"[1944-12-31, 1981-12-31, 1994-12-31, 2006-12-3...",Open Data Zurich,,,"[Sicherheitsdepartement, Stadtpolizei Zürich, ...","[City, Oerlikon, Unterstrass, Enge, Schwamendi...","[Verpflegungsbetriebe, Horte]","[1944-12-31, 1981-12-31, 1994-12-31, 2006-12-3...","Arbeitsstätten nach Betriebsart, Raum, Zeit",[2018-08-08],https://ld.stadt-zuerich.ch/statistics/dataset...,
1,[],Datenaggregat,,,,"[2014-06-30, 1986-06-30, 1980-06-30, 1972-06-3...",Open Data Zurich,,,"[Schul- und Sportdepartement, Schulamt]",[Stadt Zürich (ab 1934)],"[Geschlecht, Horte]","[2014-06-30, 1986-06-30, 1980-06-30, 1972-06-3...","Beschäftigte nach Betriebsart, Raum, Geschlech...",[2018-08-08],https://ld.stadt-zuerich.ch/statistics/dataset...,
2,"[Bildung, Kultur & Sport]",Datenaggregat,,,,"[2014-06-30, 1986-06-30, 1980-06-30, 1972-06-3...",Open Data Zurich,,,"[Zentralbibliothek Zürich, Schul- und Sportdep...",[Stadt Zürich (ab 1934)],"[, Horte]","[2014-06-30, 1986-06-30, 1980-06-30, 1972-06-3...","Beschäftigte nach Betriebsart, Raum, Zeit","[2018-08-08, 2018-08-21]",https://ld.stadt-zuerich.ch/statistics/dataset...,
3,[],Datenaggregat,,,,"[1986-06-30, 1980-06-30, 1972-06-30, 1978-06-3...",Open Data Zurich,,,"[Schul- und Sportdepartement, Schulamt]",[Stadt Zürich (ab 1934)],"[Horte, Heimatland]","[1986-06-30, 1980-06-30, 1972-06-30, 1978-06-3...","Zuschauer/innen, Besucher/innen nach Betriebsa...",[2018-08-08],https://ld.stadt-zuerich.ch/statistics/dataset...,
4,[],Datenaggregat,,,,"[2014-06-30, 1986-06-30, 1980-06-30, 1972-06-3...",Open Data Zurich,,,"[Schul- und Sportdepartement, Schulamt]",[Stadt Zürich (ab 1934)],"[Geschlecht, Horte]","[2014-06-30, 1986-06-30, 1980-06-30, 1972-06-3...","Zuschauer/innen, Besucher/innen nach Betriebsa...",[2018-08-08],https://ld.stadt-zuerich.ch/statistics/dataset...,
